# Test API

To test the API submission process

In [ ]:
import traceback
import numpy as np
import pandas as pd
#import datatable as dt
import gc
import time
import os
import xgboost as xgb
import talib as ta
from script.crypto_API import *

## Model Version

In [ ]:
ASSET_DETAILS_CSV = './data/asset_details.csv'
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")

version = 58
retrained = True #set to False to get correct out-train score
pre_minute=200 #psets['adx']*2+1 #for smoothing sensitive indicators
if version < 46:    
    if retrained:
        MOD_FOLDER = "./trainedXGB/retrained" + f"/model_nof_{version}_sub_sub"
    else:
        MOD_FOLDER = "./trainedXGB" + f"/model_nof_{version}"
else:
    MOD_FOLDER = "./trainedXGB" + f"/model_nof_{version}"

MOD_FOLDER

In [ ]:
from os.path import exists
def model_reload_train():
    print(f"model from {MOD_FOLDER}")
    models = {}
    for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
        if version <46:
            model_file = MOD_FOLDER + f"/model_{asset_id}.json"
        elif retrained:
            model_file = MOD_FOLDER + f"/model_{asset_id}_alldata.json"
        else:
            model_file = MOD_FOLDER + f"/model_{asset_id}.json"

        if exists(model_file):
            model = xgb.Booster()
            model.load_model(model_file)
            models[asset_id] = model
    return models

models = model_reload_train()
models[0].feature_names

In [ ]:
xgb.plot_importance(models[2],max_num_features=20)

## get_features

In [79]:
# psets34 = dict(zip(['lrtn','fastk1','fastk2','adx','macd_s','macd_l','rsi','std_Crypto_Index','std_lr_15','std_Mkt_lrt_15'],
#                    [15, 15, 5, 30, 5, 25, 60, 5, 15, 30]))
      
import pickle
with open(MOD_FOLDER+f"/feature_best{version}", "rb") as f:
    psets=pickle.load(f)
    
if version < 44:
    psets['vol_sum'] = 0
if 'rsi' not in psets:
    psets['rsi'] = None

psets

{'vol_sum': 15,
 'std_lr_15': 240,
 'std_Mkt_lrt_15': 10,
 'std_Crypto_Index': 30,
 'rsi': 40,
 'macd_sig': 15,
 'macd_s': 15,
 'macd_l': 40,
 'lrtn': 30,
 'fastk2': 30,
 'fastk1': 15,
 'beta_s': '6h',
 'beta_l': '2d',
 'adx': 40}

In [102]:
def log_return(series, periods=5):
    return np.log(series).diff(periods)


def lag_featuresShort(df, fastk1,fastk2,adx,macd_s,macd_l,macd_sig,rsi,vol_sum,std_Crypto_Index,std_lr_15,std_Mkt_lrt_15, **kwargs):    
    if vol_sum >0 :
        df[f'vol_sum_{vol_sum}'] = ta.SMA(df['Volume'],vol_sum)*vol_sum
    ####TECH indicators
    df['slowK'], df['slowD'] = ta.STOCH(df.High, df.Low, df.Close, 
                                        fastk_period=fastk1, slowk_period=int(3*fastk1/5), slowd_period=int(3*fastk1/5),
                                        slowk_matype=0, slowd_matype=0)
    df['fastK'], df['fastD'] = ta.STOCHF(df.High, df.Low, df.Close,
                                         fastk_period=fastk2, fastd_period=int(3*fastk2/5), 
                                         fastd_matype=0)
    df[f'macd_{macd_s}_{macd_l}'],df[f'macd_signal_{macd_sig}'], df['macd_hist'] = \
                ta.MACD(df['Close'],fastperiod=macd_s, slowperiod=macd_l, signalperiod=macd_sig)
    ###std volatility
    #df[f'std_lr_15_{std_lr_15}'] = ta.STDDEV(df.lr_15,timeperiod=std_lr_15, nbdev=1)
    df[f'std_Mkt_lrt_15_{std_Mkt_lrt_15}'] = ta.STDDEV(df.Mkt_lrt_15,timeperiod=std_Mkt_lrt_15, nbdev=1)
    df[f'std_Crypto_Index_{std_Crypto_Index}'] = ta.STDDEV(df.Crypto_Index,timeperiod=std_Crypto_Index, nbdev=1)
    
def lag_featuresLong(df, fastk1,fastk2,adx,macd_s,macd_l,macd_sig,rsi,vol_sum,std_Crypto_Index,std_lr_15,std_Mkt_lrt_15, **kwargs):    
    ##smoothing sensitive ADX 150, RSI250, AD unKNOWN
    if rsi is not None:
        df[f'rsi_{rsi}'] = ta.RSI(df['Close'], timeperiod=rsi)
    df[f'adx_{adx}'] = ta.ADX(df['High'], df['Low'],df['Close'], timeperiod=adx)
    #df['AD'] = ta.AD(df['High'], df['Low'],df['Close'], df['Volume'])
    #####new after mod 49
    # if 'ATR' in kwargs:       
    #     df[f"ATR_{kwargs['ATR']}"] = ta.ATR(df['High'], df['Low'],df['Close'], timeperiod=kwargs['ATR'])
    #     df['TRENDLINE'] =ta.HT_TRENDLINE(df['Open'])
    #     df[f"willr_{kwargs['willr']}"] = ta.WILLR(df['High'], df['Low'],df['Close'], timeperiod=kwargs['willr'])
    
def beta_resid(df): 
    b = ((ta.MULT(df.Mkt_lrt_15,df.lr_15).mean())/ \
        (ta.MULT(df.Mkt_lrt_15,df.Mkt_lrt_15).mean()))
    if b in [np.nan,np.inf,-np.inf]:
        b=0
    return b 

def get_featuresShort(df_feat, psets):
    pd.options.mode.chained_assignment = None  # default='warn'
    df_feat[f"lr_15_resid_{psets['beta_s']}"] = ta.SUB(df_feat.lr_15, ta.MULT(df_feat[f"beta_{psets['beta_s']}"], df_feat.Mkt_lrt_15)).rename(f"lr_15_resid_{psets['beta_s']}")
    df_feat[f"lr_15_resid_{psets['beta_l']}"] = ta.SUB(df_feat.lr_15, ta.MULT(df_feat[f"beta_{psets['beta_l']}"], df_feat.Mkt_lrt_15)).rename(f"lr_15_resid_{psets['beta_l']}")
    df_feat[f"lrtn_index_{psets['lrtn']}"] = log_return(df_feat.Crypto_Index, psets['lrtn'])
    lag_featuresShort(df_feat, **psets)
    return df_feat

def get_featuresLong(df_feat, psets):
    pd.options.mode.chained_assignment = None  # default='warn'
    lag_featuresLong(df_feat, **psets)
    return df_feat.iloc[:,-2:]

## API submission

In [ ]:
pd.to_datetime(1632182400,unit="s",
               infer_datetime_format=True)

In [ ]:
start = datestring_to_timestamp('2021-06-13T00:00:00')#1623542400
#end = datestring_to_timestamp('2021-09-21T00:00:00')#1632182400
end = datestring_to_timestamp('2021-06-13T05:06:00') #1623560760
train_df = read_csv_slice(file_path='./data/train.csv', 
                          use_window=[start, end])


In [81]:
def beta_window(beta):
    num, unit = int(beta[:-1]),beta[-1]
    if unit == 'h':
        width = 60*num
    elif unit == 'd':
        width = 60*24*num
    return width

beta_sw = beta_window(psets['beta_s'])
beta_lw = beta_window(psets['beta_l'])
beta_lw

2880

In [82]:
pre_minute_beta =  beta_lw + 15#for beta calculations
pre_minute, pre_minute_beta

(200, 2895)

In [158]:
add_weight_map = dict(zip(df_asset_details.Asset_ID, 
                        df_asset_details.Weight/df_asset_details.Weight.sum()))

###load sup_train
sup_train = pd.read_csv('./data/supplemental_train.csv')
sup_train = sup_train.sort_values('timestamp').set_index("timestamp")
ind = sup_train.index.unique()
###consistent timestamp for all 14 assets
def reindex(df):
    df = df.reindex(range(ind[0],ind[-1]+60,60),method='nearest')
    df = df.fillna(method="ffill").fillna(method="bfill")
    return df
sup_train = sup_train.groupby('Asset_ID').apply(reindex).reset_index(0, drop=True).sort_index()
sup_train = sup_train.iloc[(-14*pre_minute_beta):,:]
#add weight
sup_train['Weight'] = sup_train['Asset_ID'].map(add_weight_map)
sup_train.drop('Target',axis=1, inplace=True)
sup_train.set_index('Asset_ID',append=True, inplace=True)
#######################################add lr_15,mkt_lr_15,crypto_index, beta,lr_mkt_resid
lr_15 = sup_train.groupby('Asset_ID').apply( 
        lambda x: log_return(x[['Close']],15)
        )
sup_train['lr_15'] = lr_15['Close']

mkt_lr_15 = sup_train.groupby('timestamp').apply( 
    lambda x: x[["lr_15", "Close"]].multiply(x["Weight"], axis="index").sum()
    )
mkt_lr_15.columns = ['Mkt_lrt_15','Crypto_Index']
firsts = sup_train.index.get_level_values('timestamp')
sup_train[['Mkt_lrt_15','Crypto_Index']] = mkt_lr_15.loc[firsts].values

####make beta,lr_mkt_resid, std, placeholder
sup_train[f"beta_{psets['beta_s']}"] = 0
sup_train[f"beta_{psets['beta_l']}"] = 0
sup_train[f"std_lr_15_{psets['std_lr_15']}"] = 0
sup_train.iloc[-14:,:]

Count          Open          High           Low  \
timestamp  Asset_ID                                                     
1642982400 6         2510.0   2540.678571   2543.100000   2533.960000   
           11          40.0    156.764000    156.930000    156.200000   
           10          25.0   1833.565000   1834.700000   1831.050000   
           9          214.0    112.293000    112.470000    111.840000   
           8           49.0      0.814567      0.815300      0.811200   
           7          280.0     25.128433     25.191000     24.984000   
           5          468.0      2.272950      2.277700      2.260000   
           4          227.0      0.141784      0.141900      0.141300   
           3          457.0      1.124904      1.126200      1.119907   
           2          139.0    302.506000    302.770000    301.800000   
           1         2917.0  36262.038571  36302.000000  36176.450000   
           0          420.0    383.901333    384.003000    382.371000   
           12         492.0      0.200940      0.201066      0.199352   
           13         235.0      0.057520      0.057618      0.057389   

                            Close        Volume          VWAP    Weight  \
timestamp  Asset_ID                                                       
1642982400 6          2536.151429  1.057766e+03   2538.323926  0.144188   
           11          156.550000  7.439747e+01    156.638720  0.039370   
           10         1833.018333  4.190290e+00   1833.173692  0.026874   
           9           111.983333  1.243860e+03    112.127619  0.058657   
           8             0.812400  6.019270e+03      0.813656  0.026874   
           7            25.028067  6.709816e+03     25.093705  0.050867   
           5             2.264800  2.021940e+05      2.269373  0.033911   
           4             0.141391  1.009688e+06      0.141649  0.086971   
           3             1.121151  2.499232e+05      1.123218  0.107797   
           2           302.112000  2.096457e+02    302.422931  0.058657   
           1         36221.987143  1.100532e+02  36247.575361  0.165850   
           0           382.930000  1.490571e+03    383.253768  0.105286   
           12            0.200093  2.983796e+06      0.200413  0.050867   
           13            0.057478  8.988253e+05      0.057519  0.043830   

                        lr_15  Mkt_lrt_15  Crypto_Index  beta_6h  beta_2d  \
timestamp  Asset_ID                                                         
1642982400 6         0.000148   -0.001114   6494.648269        0        0   
           11       -0.000843   -0.001114   6494.648269        0        0   
           10       -0.002008   -0.001114   6494.648269        0        0   
           9        -0.001254   -0.001114   6494.648269        0        0   
           8        -0.001189   -0.001114   6494.648269        0        0   
           7        -0.004492   -0.001114   6494.648269        0        0   
           5        -0.002785   -0.001114   6494.648269        0        0   
           4        -0.000066   -0.001114   6494.648269        0        0   
           3        -0.001616   -0.001114   6494.648269        0        0   
           2         0.001066   -0.001114   6494.648269        0        0   
           1        -0.001337   -0.001114   6494.648269        0        0   
           0        -0.001288   -0.001114   6494.648269        0        0   
           12       -0.002306   -0.001114   6494.648269        0        0   
           13       -0.000622   -0.001114   6494.648269        0        0   

                     std_lr_15_240  
timestamp  Asset_ID                 
1642982400 6                     0  
           11                    0  
           10                    0  
           9                     0  
           8                     0  
           7                     0  
           5                     0  
           4                     0  
           3                     0  
           2

In [159]:
myapi = API(train_df)
from datetime import datetime 
start_time = datetime.now()

for df_test, df_pred in myapi:
    num_asset_test = df_test.shape[0]
    row_asset_id_map = dict(zip(df_test.row_id, df_test.Asset_ID))
    test_timestamp = df_test.timestamp.values[0]
    print(f"{test_timestamp}", end = "\r")
    
    timestamp_list = sup_train.index.get_level_values('timestamp').unique().values
    timestamp_list = np.append(timestamp_list,test_timestamp)
    #######################################format df_test
    ###add weight and index
    df_test['Weight'] = df_test['Asset_ID'].map(add_weight_map)
    ###fillin missing assets as nan
    df_test.set_index(['timestamp','Asset_ID'],inplace=True)
    df_test = df_test.reindex(list(zip([test_timestamp]*14,range(14))))
    ########################################concat to sup_train, add lr_15,mkt_lr_15,crypto_index
    sup_train = pd.concat([sup_train,df_test.drop('row_id',axis=1)],join='outer')
    #########################################fill in missing assets as forward
    if num_asset_test <14:
        #ffill in missing
        sup_train = sup_train.groupby('Asset_ID').apply(lambda x: x.fillna(method="ffill")).iloc[14:,:]
    else:
        sup_train = sup_train.iloc[14:,:]
    
    test_lr_15 = sup_train.loc[timestamp_list[[-16,-1]]].groupby('Asset_ID').apply(
        lambda x: np.log(x[['Close']]).diff()
    )
    sup_train.loc[test_timestamp, 'lr_15'] = test_lr_15.loc[test_timestamp,'Close'].values
    sup_train.loc[test_timestamp, ['Mkt_lrt_15','Crypto_Index']] = sup_train.loc[test_timestamp, ["lr_15", "Close"]].multiply(sup_train.loc[test_timestamp,"Weight"], axis="index").sum(skipna=True).values
    
    ########################################beta_sl
    beta_s = sup_train[['lr_15','Mkt_lrt_15']].iloc[-14*(beta_sw):,:].groupby('Asset_ID').apply(
        lambda x: beta_resid(x)
    ).rename(f"beta_{psets['beta_s']}")
    beta_l = sup_train[['lr_15','Mkt_lrt_15']].iloc[-14*(beta_lw):,:].groupby('Asset_ID').apply(
        lambda x: beta_resid(x)
    ).rename(f"beta_{psets['beta_l']}")
    sup_train.loc[test_timestamp, [f"beta_{psets['beta_s']}",f"beta_{psets['beta_l']}"]] = \
        pd.concat([beta_s, beta_l],axis=1).values
    #######################################add features to test timestamp
    long_std = sup_train.iloc[-14*psets['std_lr_15']:,:].groupby('Asset_ID').apply(lambda x: x.lr_15.std())
    sup_train.loc[test_timestamp, f"std_lr_15_{psets['std_lr_15']}"] = long_std.values
    #########################################adx rsi
    # adxrsi = sup_train.iloc[(-14*pre_minute):,:].groupby('Asset_ID').apply(lambda x: make_AdxRsi(x,psets['adx'],psets['rsi']))
    # sup_train.loc[test_timestamp, [f"adx_{psets['adx']}",f"rsi_{psets['rsi']}"]] = adxrsi.loc[test_timestamp].values
    sup_train2 = sup_train.iloc[(-14*pre_minute):,:].copy()
    
    xx_testShort=sup_train2.iloc[(-14*54):,:].groupby('Asset_ID').apply(
        lambda x: get_featuresShort(x,psets)
    ).loc[test_timestamp]
    xx_testLong=sup_train2.groupby('Asset_ID').apply(
        lambda x: get_featuresLong(x,psets)
    ).loc[test_timestamp]
    xx_test = pd.concat([xx_testShort, xx_testLong], axis = 1)
    #rdy for prediction
    y_pred=df_test.apply(lambda row: models[row.name[1]].predict(
                            xgb.DMatrix(pd.DataFrame([xx_test.loc[row.name[1],models[row.name[1]].feature_names]]))
                                                                )[-1]
                         ,axis =1)
    #match with row_id
    y_pred.reset_index('timestamp',drop=True,inplace=True)
    df_pred['Target']= y_pred.loc[df_pred['row_id'].map(row_asset_id_map)].values
    myapi.predict(df_pred)
    

time_elapsed = datetime.now() - start_time
print('Time elapsed total (hh:mm:ss.ms) {}'.format(time_elapsed))
print(f'time elapsed per iteration {time_elapsed/myapi.init_num_times}')
print(f'Submission time estimate {129600*time_elapsed/myapi.init_num_times}')

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set. ;)
Time elapsed total (hh:mm:ss.ms) 0:00:24.832236
time elapsed per iteration 0:00:00.081151
Submission time estimate 2:55:17.182306


In [171]:
sup_train2.iloc[(-14*180):,:].groupby('Asset_ID').apply(
        lambda x: get_featuresShort(x,psets)
    ).loc[test_timestamp].iloc[:,8:]

,lr_15,Mkt_lrt_15,Crypto_Index,beta_6h,beta_2d,std_lr_15_240,lr_15_resid_6h,lr_15_resid_2d,lrtn_index_30,vol_sum_15,slowK,slowD,fastK,fastD,macd_15_40,macd_signal_15,macd_hist,std_Mkt_lrt_15_10,std_Crypto_Index_30
Asset_ID,,,,,,,,,,,,,,,,,,,
0,-0.000206,-0.00207,6336.023324,-0.404653,-0.211958,0.003541,-0.001044,-0.000645,0.005257,2.116159e+04,49.706745,69.982001,54.760031,71.601353,0.208698,0.218049,-0.009350,0.002402,19.404595
1,0.002048,-0.00207,6336.023324,-0.070057,0.028896,0.003426,0.001903,0.002108,0.005257,1.444397e+03,64.594840,75.922817,70.064886,80.380509,66.508013,58.716951,7.791062,0.002402,19.404595
2,-0.003252,-0.00207,6336.023324,2.441841,2.227988,0.003426,0.001804,0.001361,0.005257,2.315159e+03,31.084823,59.207161,18.010204,51.492372,-0.587957,-0.391471,-0.196486,0.002402,19.404595
3,-0.006541,-0.00207,6336.023324,1.036110,1.088726,0.003783,-0.004396,-0.004287,0.005257,3.888636e+06,32.525770,55.842141,12.682224,57.120105,-0.001268,-0.000052,-0.001216,0.002402,19.404595
4,0.000748,-0.00207,6336.023324,2.942204,2.674791,0.003000,0.006839,0.006286,0.005257,1.299010e+07,55.665945,69.508423,48.699634,68.200170,0.000292,0.000288,0.000005,0.002402,19.404595
5,-0.006080,-0.00207,6336.023324,2.816035,2.555796,0.003737,-0.000250,-0.000789,0.005257,5.651365e+05,36.977630,63.723914,26.558986,59.084130,-0.005804,-0.001266,-0.004538,0.002402,19.404595
6,-0.002059,-0.00207,6336.023324,-0.246754,-0.072703,0.004801,-0.002570,-0.002210,0.005257,1.588374e+04,44.044861,63.743895,33.185744,61.401912,0.095480,0.707931,-0.612451,0.002402,19.404595
7,-0.002487,-0.00207,6336.023324,2.933348,2.681796,0.004445,0.003586,0.003066,0.005257,3.695120e+04,43.941315,64.990395,28.585879,59.933046,-0.014687,0.004908,-0.019596,0.002402,19.404595
8,-0.007288,-0.00207,6336.023324,0.763681,0.822941,0.008080,-0.005707,-0.005584,0.005257,1.555626e+05,45.128259,52.243592,42.984190,56.220388,0.001361,0.003066,-0.001705,0.002402,19.404595


In [170]:
sup_train2.iloc[(-14*200):,:].groupby('Asset_ID').apply(
        lambda x: get_featuresShort(x,psets)
    ).loc[test_timestamp].iloc[:,8:]

,lr_15,Mkt_lrt_15,Crypto_Index,beta_6h,beta_2d,std_lr_15_240,lr_15_resid_6h,lr_15_resid_2d,lrtn_index_30,vol_sum_15,slowK,slowD,fastK,fastD,macd_15_40,macd_signal_15,macd_hist,std_Mkt_lrt_15_10,std_Crypto_Index_30
Asset_ID,,,,,,,,,,,,,,,,,,,
0,-0.000206,-0.00207,6336.023324,-0.404653,-0.211958,0.003541,-0.001044,-0.000645,0.005257,2.116159e+04,49.706745,69.982001,54.760031,71.601353,0.208761,0.218146,-0.009385,0.002402,19.404595
1,0.002048,-0.00207,6336.023324,-0.070057,0.028896,0.003426,0.001903,0.002108,0.005257,1.444397e+03,64.594840,75.922817,70.064886,80.380509,66.505629,58.713231,7.792397,0.002402,19.404595
2,-0.003252,-0.00207,6336.023324,2.441841,2.227988,0.003426,0.001804,0.001361,0.005257,2.315159e+03,31.084823,59.207161,18.010204,51.492372,-0.587951,-0.391462,-0.196489,0.002402,19.404595
3,-0.006541,-0.00207,6336.023324,1.036110,1.088726,0.003783,-0.004396,-0.004287,0.005257,3.888636e+06,32.525770,55.842141,12.682224,57.120105,-0.001268,-0.000053,-0.001215,0.002402,19.404595
4,0.000748,-0.00207,6336.023324,2.942204,2.674791,0.003000,0.006839,0.006286,0.005257,1.299010e+07,55.665945,69.508423,48.699634,68.200170,0.000292,0.000287,0.000005,0.002402,19.404595
5,-0.006080,-0.00207,6336.023324,2.816035,2.555796,0.003737,-0.000250,-0.000789,0.005257,5.651365e+05,36.977630,63.723914,26.558986,59.084130,-0.005803,-0.001264,-0.004538,0.002402,19.404595
6,-0.002059,-0.00207,6336.023324,-0.246754,-0.072703,0.004801,-0.002570,-0.002210,0.005257,1.588374e+04,44.044861,63.743895,33.185744,61.401912,0.094040,0.705684,-0.611645,0.002402,19.404595
7,-0.002487,-0.00207,6336.023324,2.933348,2.681796,0.004445,0.003586,0.003066,0.005257,3.695120e+04,43.941315,64.990395,28.585879,59.933046,-0.014672,0.004933,-0.019605,0.002402,19.404595
8,-0.007288,-0.00207,6336.023324,0.763681,0.822941,0.008080,-0.005707,-0.005584,0.005257,1.555626e+05,45.128259,52.243592,42.984190,56.220388,0.001361,0.003065,-0.001704,0.002402,19.404595


In [160]:
xx_test.loc[xx_test.isin([np.nan]).any(axis=1)]

,Count,Open,High,Low,Close,Volume,VWAP,Weight,lr_15,Mkt_lrt_15,...,slowD,fastK,fastD,macd_15_40,macd_signal_15,macd_hist,std_Mkt_lrt_15_10,std_Crypto_Index_30,rsi_40,adx_40
Asset_ID,,,,,,,,,,,,,,,,,,,,,


In [161]:
##scoring
id_2_weight = dict(zip(df_asset_details.Asset_ID, df_asset_details.Weight))
df, score = myapi.score(id_2_weight)
print(f"Your LB score is {round(score, 4)}")

Your LB score is 0.0405


In [162]:
myapi.predictions

[            row_id    Target
 22221694  22221694 -0.002198
 22221695  22221695 -0.007330
 22221696  22221696  0.002823
 22221697  22221697 -0.000791
 22221698  22221698  0.006418
 22221699  22221699  0.001696
 22221700  22221700 -0.033926
 22221701  22221701  0.002050
 22221702  22221702 -0.004953
 22221703  22221703 -0.008457
 22221704  22221704 -0.002458
 22221705  22221705  0.004139
 22221706  22221706 -0.014239
 22221707  22221707 -0.007991,
             row_id    Target
 22221708  22221708 -0.003254
 22221709  22221709 -0.008519
 22221710  22221710  0.003240
 22221711  22221711 -0.000828
 22221712  22221712  0.005308
 22221713  22221713 -0.004081
 22221714  22221714 -0.026277
 22221715  22221715  0.001878
 22221716  22221716 -0.005321
 22221717  22221717 -0.007143
 22221718  22221718 -0.001378
 22221719  22221719  0.002223
 22221720  22221720 -0.014982
 22221721  22221721 -0.008789,
             row_id    Target
 22221722  22221722 -0.003605
 22221723  22221723 -0.008005
 2222172